<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />MSc. Carsten Bauer</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 6</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2020</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml)

**Themen dieses Notebooks:** Zeitaufgelöste Streuung an einem Potential, "Split-step" Methode, Fourier-Transformationen

## Zeitaufgelöste Streuung an einem Potential

Bevor es richtig losgeht, binden wir wie immer einige Pakete ein. Insbesondere nutzen wir das "[FFTW](https://en.wikipedia.org/wiki/FFTW)" Paket zur Berechnung der (diskreten) **Fourier-Transformationen** der Wellenfunktion zwischen Orts- und Impulsraumdarstellung.

In [1]:
# Pakete einbinden
using PyPlot
using LinearAlgebra
using FFTW # Fourier-Transformation
pygui(true);

Für die Integration der Schrödingergleichung **diskretisieren** wir die Zeitentwicklung in Schritten von $dt$. Auch die Koordinaten in Orts- und im Fourier-transformierten Impulsraum diskretisieren wir, wobei wir wir $N=2^{11}$ diskrete Koordinaten betrachten wollen. 

In [2]:
# Diskretisierungen
# ħ = m = 1.0 per Definition

dt = 0.01
N = 2^11

dx = 0.1
xs = dx .* ((1:N) .- N/2)

dk = 2π / (N*dx)
ks = dk .* ((1:N) .- N/2);

### Definition eines einlaufenden Wellenpakets ###

Als erstes wollen wir ein quantenmechanisches Wellenpaket definieren, welches im Ortsraum anfänglich die Form einer **Gauss'schen Glockenkurve** haben soll und sich mit einem Impuls $k_0$ nach rechts (oder links) bewegen soll.

$\quad \quad \psi(x) = \frac{1}{\sqrt{\sigma \cdot \sqrt{\pi}}} \cdot e^{-\frac{\left(x-x_0\right)^2}{2\sigma^2} + \, i \, x \cdot k_0}$



In [36]:
# Definition der Wellenfuntion
function set_wavefunction(k0, xs)
    # Gaussian
    σ = 12.0 / k0
    
    # "linkes" Wellenpaket
    x0 = -60
    ψ_x0 = @. 1/sqrt(σ * sqrt(π)) * exp(-((xs - x0)/σ)^2/2 + im * xs * k0)
    
    # "rechtes" Wellenpaket
#    x1 = 60
#    ψ_x1 = @. 1/sqrt(σ * sqrt(π)) * exp(-((xs - x1)/σ)^2/2 - im * xs * k0)

    return ψ_x0 # + ψ_x1
end

set_wavefunction (generic function with 1 method)

### Definition des Potentials ###

Im folgenden wollen wir ein Potential definieren, an dem wir das einlaufende Wellenpaket streuen wollen. Dabei betrachten wir Potentialbarrieren (positives Potential),  Potentialtöpfe (negatives Potential), und Doppelbarrieren in einem gewissen Abstand. 

In [19]:
# Definition des Potential
function set_potential(V0, V_width, V_distance, N, xs)
    V = zeros(N)
    
    ### einzelne(r) Potentialbarriere oder -topf
    if V_distance == 0
        for i in 1:N
            if abs(xs[i]) < V_width/2
                V[i] = V0
            end
        end
    end
    
    ### doppelte Potentialbarriere
    if abs(V_distance) > 0
        for i in 1:N
            if (abs(xs[i]) < V_distance/2 + V_width) && (abs(xs[i]) > V_distance/2)
                V[i] = V0
            end
        end
    end
        
    return V
end

set_potential (generic function with 1 method)

### Halb-Schritt Integration der zeitabhängigen Schrödinger-Gleichung ###

Die eigentliche Hauptroutine unseres Programms formuliert das gerade in der Tafelpräsentation eingeführte Halb-Schritt-Verfahren zur Integration der zeitabhängigen Schrödinger-Gleichung. 

Dabei wechseln wir mittel Fourier-Transformation geschickt zwischen Orts- und Impulsraum-Darstellung hin und her, und integrieren jeweils den "einfachen" Teil der Schrödingergleichung.

Schrödingergleichung im **Ortsraum**

$\quad \quad \left(-\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2} + V(x) \right)\psi(x,t) = i\hbar \frac{\partial}{\partial t}\psi(x,t)$

Schrödingergleichung im **Impulsraum**

$\quad \quad \left(-\frac{\hbar^2k^2}{2m} + V\left(i \frac{\partial}{\partial k}\right) \right)\tilde\psi(k,t) = i\hbar \frac{\partial}{\partial t}\tilde\psi(k,t)$

In der numerischen Umsetzung der Fourier-Transformation werden wir mit einer *diskreten* Fourier-Transformation arbeiten (eben jenem [FFTW](https://en.wikipedia.org/wiki/FFTW)-Verfahren). Dazu müssen wir die Wellenfunktion in Orts- und Impulsraum in diskreter Form darstellen. Eine gute Einführung hierzu findet sich etwa in diesem [Blog-Post](https://jakevdp.github.io/blog/2012/09/05/quantum-python/) zum hier beschriebenen Halb-Schritt-Verfahren.


In [34]:
# Halb-Schritt Integration
function split_step_integration(ψ_x0, V, xs, dx, ks, dt)

    # diskrete Fourier-Darstellung im Ortsraum
    ψ_mod_x = @. ψ_x0 * exp(- im * ks[1] * xs) * dx / (sqrt(2π))

    ### Iteration des Halb-Schritt-Verfahrens 
    step  = 1
    while step < 2000 && (abs.(ψ_x0).^2)[1] < 0.001 && (abs.(ψ_x0).^2)[end] < 0.001
        # update step
        for j in 1:20
            # (halber) Intergrationsschritt im Ortsraum
            @. ψ_mod_x *= exp(- im * 0.5 * V * dt)
            # Fourier-Transformation in den Impulsraum
            ψ_mod_k = fft(ψ_mod_x) 
            # (ganzer) Intergrationsschritt im Impulsraum
            @. ψ_mod_k *= exp(- im * 0.5 * (ks * ks) * dt) 
            # Fourier-Transformation zurück in den Ortsraum
            ψ_mod_x = ifft(ψ_mod_k)  
            # (halber) Intergrationsschritt im Ortsraum
            @. ψ_mod_x *= exp(- im * 0.5 * V * dt) 
        end
        
        # diskrete Fourier-Darstellung im Ortsraum
        ψ_x0 = @. ψ_mod_x * exp(im * ks[1] * xs) * sqrt(2π) / dx

        # update Plot 
        ψ_plot.set_data(xs, abs.(ψ_x0).^2)

        # delay for plotting
        pause(0.001)
        step += 1
    end    
end

split_step_integration (generic function with 1 method)

### Simulation ###

Damit haben wir alles was wir brauchen und können loslegen. Betrachten wir dazu eine Reihe von unterschiedlich parametrisierten Beispielen.

In [37]:
# Initialisierungen der Parameter
k0 = 1.5
V0 = 1.0
V_width = 5.0
V_distance = -10.0

V = set_potential(V0, V_width, V_distance, N, xs)
ψ_x0 = set_wavefunction(k0, xs)


# Display
fig = figure()
fig.canvas.manager.window.move(5,5)
gcf().set_size_inches(10, 7)
gcf().clf()
gcf().set_facecolor("lightgray")
    
# Limits
ylim(-0.03, 0.28)
xlim(-100, 100)
axis("off")

# Plotten der Anfangskonfiguration    
line_plot = plot(xs, zeros(length(xs)), color="black",   linewidth=3)
V_plot    = plot(xs, V,                 color="blue",    linewidth=1)
fill_between(xs, V,color="lightblue")
ψ_plot    = plot(xs, abs.(ψ_x0).^2,     color="#850000", linewidth=2)[1]

readline();

split_step_integration(ψ_x0, V, xs, dx, ks, dt)


stdin> 
